In [1]:
#execute this file after you start the experiment
#function:monitor and record the overhead info of fuzzer container

In [2]:
import re
import subprocess
#function:get all of the fuzzer containers
#attention:use your own running environment to replace "myruntwo"
def get_fuzzer_container_list():
    containers = []
    p = subprocess.Popen("docker ps -a | grep 'myruntwo' | grep -v 'framework' | grep -v 'driver'", shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    for linetmp in p.stdout.readlines():
        linetmpstr = str(linetmp, encoding = "utf8")
        if "myruntwo" in linetmpstr:
            container = re.findall(r"(myruntwo_.*_\d+)", linetmpstr)
            containers.append(container[0])
    return containers
print(get_fuzzer_container_list())

[]


In [3]:
def parse_kbmbgb_to_mb(str_capacity):
    capacity = float(re.findall(r"(\d*.*\d+)(kB|MiB|GiB|MB|GB|KB)", str_capacity)[0][0])
    if "kB" in str_capacity or "KB" in str_capacity:
        capacity = capacity / 1000
    elif "GiB" in str_capacity or "GB" in str_capacity:
        capacity = capacity * 1000
    print(capacity)
    return capacity

In [4]:
from onefuzzer.fscve_step1_db import OverheadRecord, session_overhead


#info sample
# container_str="470cafd45268   myruntwo_afl-0_1        101.02%   102MiB / 15.28GiB     0.65%     590kB / 112MB   48.4MB / 134MB   5"
def parse_and_save_container_info(container_str:str):
    global record_time
    container_item = re.split('[ /]+',container_str)
    print(container_item)
    if len(container_item)!=11:
        print("error!! not able to parse the info!Please check it !")
        return
    container_name = container_item[1]

    cpu_usage =  float(re.findall(r"(.+)%", container_item[2])[0])
    # print(cpu_usage)


    mem_used = parse_kbmbgb_to_mb(container_item[3])
    mem_total = parse_kbmbgb_to_mb(container_item[4])
    mem_usage = float(re.findall(r"(.+)%", container_item[5])[0])

    netin = parse_kbmbgb_to_mb(container_item[6])
    netout = parse_kbmbgb_to_mb(container_item[7])

    diskin = parse_kbmbgb_to_mb(container_item[8])
    diskout = parse_kbmbgb_to_mb(container_item[9])

    pids = int(container_item[10])

    new_record = OverheadRecord(container_name = container_name,cpu_usage = cpu_usage,mem_used = mem_used,mem_total = mem_total,mem_usage = mem_usage,netin = netin,netout = netout,diskin = diskin,diskout = diskout,pids = pids,record_time=record_time )
    session_overhead.add(new_record)
    session_overhead.commit()

In [5]:
#function:get the resource overhead of each fuzzer container nce mannually
import subprocess
def get_overhead_once():
    global record_time
    fuzzer_containers = get_fuzzer_container_list()
    cmd = "docker stats --no-stream "
    for container in fuzzer_containers:
        cmd += container + " "
    # print(cmd)
    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    for line in p.stdout.readlines():
        line_str = str(line,encoding = "utf8")
        if "CONTAINER" in line_str and "ID" in line_str:
            continue
        parse_and_save_container_info(line_str)
    record_time +=1

# get_overhead_once()

In [6]:
# coding:utf8
import time
import sched

#function:get the resource overhead of each fuzzer container periodly：default 600seconds

s = sched.scheduler(time.time, time.sleep)

def perform(inc):
    s.enter(inc, 0, perform, (inc,))
    get_overhead_once()

record_time = 0  #global var to record units(10minutes)
if __name__ == "__main__":
    global record_time
    # drop table CrashAnalysis to store new data
    session_overhead.query(OverheadRecord).delete()
    session_overhead.commit()
    record_time = 0
    s.enter(0, 0, perform, (600,))  # every 600 seconds
    s.run()

['ac4cb4bcd3b8', 'sharp_ardinghelli', '0.00%', '8.23MiB', '15.28GiB', '0.05%', '0B', '0B', '8.67MB', '0B', '1\n']
8.23
15280.0


IndexError: list index out of range